Download the dataset and load the data

In [1]:
%pip install kagglehub

import kagglehub


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


In [ ]:
# Download the dataset
path = kagglehub.dataset_download("thedevastator/online-retail-transaction-data")

# print the path
print("Path to dataset files:", path)


100%|██████████| 8.68M/8.68M [00:03<00:00, 2.70MB/s]

Extracting files...


Path to dataset files: C:\Users\franc\.cache\kagglehub\datasets\thedevastator\online-retail-transaction-data\versions\1
